## Regression Evalution Metrics
* R-Squared (R²) - explained variance (quality of model)
* Absolute-loss(MAE)- Mean absolute error
* Squared-loss(MSE)- Mean Squared Error 
* RMS-loss(RMSE) -Root Mean Squared Error

For more details https://learn.microsoft.com/en-us/dotnet/machine-learning/resources/metrics

In [98]:
#r "nuget: Microsoft.ML, 5.0.0"
#r "nuget: Microsoft.ML.FastTree, 5.0.0"
#r "nuget: Microsoft.ML.LightGbm, 5.0.0"

using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms;

Installed Packages Microsoft.ML, 5.0.0 Microsoft.ML.FastTree, 5.0.0 Microsoft.ML.LightGbm, 5.0.0

In [99]:
public class SalaryData
{
    [LoadColumn(0)]
    public float YearsExperience;

    [LoadColumn(1)]
    public float Salary;
}

public class SalaryPrediction
{
    [ColumnName("Score")]
    public float Salary;
}

In [100]:
MLContext mlContext = new MLContext();
IDataView data = mlContext.Data.LoadFromTextFile<SalaryData>(path: "Salary_data.csv", hasHeader: true, separatorChar: ',');

In [101]:

var pipeline =mlContext.Transforms.Concatenate("Features", nameof(SalaryData.YearsExperience))
.Append(mlContext.Transforms.NormalizeMeanVariance("Features"))
.Append(mlContext.Regression.Trainers.Sdca(labelColumnName: nameof(SalaryData.Salary), featureColumnName: "Features"));

In [102]:
void DisplayData(DataDebuggerPreview preview)
{
    Console.WriteLine("\n=== Transformed Data Preview ===\n");
    // Get column names
    var columnNames = preview.Schema.Select(c => c.Name).ToArray();
    // Print headers
    foreach (var col in columnNames)
        Console.Write($"{col,-20}");
    Console.WriteLine();
    Console.WriteLine(new string('-', columnNames.Length * 20));
    // Print rows
    foreach (var row in preview.RowView)
    {
        foreach (var column in row.Values)
        {
            string value = column.Value == null ? "NULL" : column.Value.ToString();
            if (value.Length > 18) value = value.Substring(0, 17) + "..."; // shorten long vector output
            Console.Write($"{value,-20}");
        }
        Console.WriteLine();
    }
    Console.WriteLine("\n============================\n");
}

In [103]:
/* data = pipeline.Fit(data).Transform(data);
var preview = data.Preview(10);
DisplayData(preview);
*/

In [104]:
// Split and train the model
var split = mlContext.Data.TrainTestSplit(data, testFraction: 0.2);

In [105]:
// Train the model
var model=pipeline.Fit(split.TrainSet);

In [106]:
// Test the model
var prediction=model.Transform(split.TestSet);

In [107]:
Console.WriteLine("===== MODEL METRICS =====");
Console.WriteLine($"R²  : {metrics.RSquared}");
Console.WriteLine($"MAE: {metrics.MeanAbsoluteError}");
Console.WriteLine($"RMSE: {metrics.RootMeanSquaredError}");
Console.WriteLine($"MSE: {metrics.MeanSquaredError}");

===== MODEL METRICS =====
R²  : 0.9870908263787351
MAE: 2553.2109375
RMSE: 2972.8130913422574
MSE: 8837617.676055908


In [108]:
// Evualuate the model
var metrics=mlContext.Regression.Evaluate(prediction,labelColumnName:nameof(SalaryData.Salary),scoreColumnName:"Score");

In [109]:
// Predict a sample
var predEngine=mlContext.Model.CreatePredictionEngine<SalaryData,SalaryPrediction>(model);
var sampleData=new SalaryData()
{
    YearsExperience=3.2F,Salary=0F
};

var result=predEngine.Predict(sampleData);
Console.WriteLine($"Predicted Salary for {sampleData.YearsExperience} years of experience is {result.Salary}");

Predicted Salary for 3.2 years of experience is 54787.285
